# XGBoost Model for Weekly Rent

This notebook trains an XGBoost regressor to predict `rental_price` using the dataset:
`data/curated/rent_features/cleaned_listings_isochrones_added_with_best_schools.csv`.

It loads and preprocesses the data, splits into train/validation/test, fits the model,
evaluates metrics, and inspects feature importances.

In [31]:
# Imports
import os, sys
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# XGBoost
try:
    from xgboost import XGBRegressor
    try:
        # Preferred in newer XGBoost versions (>=2.x/3.x)
        from xgboost.callback import EarlyStopping as XGB_EarlyStopping
    except Exception:
        XGB_EarlyStopping = None
except Exception as e:
    print("xgboost is not installed. Install with: pip install xgboost")
    raise

np.random.seed(42)
sns.set_theme(style="whitegrid")


In [32]:
# Locate and load the dataset (supports running from repo root or notebooks/)
DATA_PATHS = [
    Path('data/curated/rent_features/cleaned_listings_isochrones_added_with_best_schools.csv'),
    Path('../data/curated/rent_features/cleaned_listings_isochrones_added_with_best_schools.csv'),
]
csv_path = None
for p in DATA_PATHS:
    if p.exists():
        csv_path = p
        break
if csv_path is None:
    raise FileNotFoundError(
        "Could not find CSV at expected paths: " + ", ".join(map(str, DATA_PATHS))
    )

# Optional sampling to speed up experimentation on large files
USE_SAMPLE = True   # set True to sample a subset for quick runs
SAMPLE_FRAC = 0.2
SAMPLE_RANDOM_STATE = 42

read_csv_kwargs = dict(low_memory=False)
df = pd.read_csv(csv_path, **read_csv_kwargs)
if USE_SAMPLE:
    df = df.sample(frac=SAMPLE_FRAC, random_state=SAMPLE_RANDOM_STATE).reset_index(drop=True)

print(f"Loaded {csv_path} with shape: {df.shape}")
df.head(3)


Loaded ../data/curated/rent_features/cleaned_listings_isochrones_added_with_best_schools.csv with shape: (2899, 318)


,property_id,rental_price,bedrooms,bathrooms,car_spaces,property_type,suburb,year,quarter,longitude,...,best_school_name_walking_10min,best_school_coord_walking_10min,best_score_walking_10min,best_dist_km_walking_10min,n_schools_walking_10min,best_school_name_walking_15min,best_school_coord_walking_15min,best_score_walking_15min,best_dist_km_walking_15min,n_schools_walking_15min
0,17346819,530.0,4,3,2,house,truganina,2025,3,144.689994,...,Bemin Secondary College,POINT (144.70975 -37.82641),0.062926,6.418188,23.0,NaN,NaN,3.000000e-08,6.245875,0.0
1,17436456,500.0,2,1,2,House,croydon-lilydale,2025,1,145.338663,...,Lilydale High School,POINT (145.34392 -37.7607),-0.000089,0.604427,2.0,Lilydale High School,POINT (145.34392 -37.7607),-8.921519e-05,0.604427,3.0
2,17736641,550.0,4,2,2,house,werribee-hoppers crossing,2025,3,144.635706,...,Werribee Secondary College,POINT (144.66874 -37.90554),-0.000060,3.374128,2.0,NaN,NaN,3.000000e-08,6.245875,2.0


In [33]:
# Target: ensure numeric and drop NAs
if 'rental_price' not in df.columns:
    raise KeyError("Column 'rental_price' not found in data.")

df['rental_price'] = pd.to_numeric(df['rental_price'], errors='coerce')
df = df.dropna(subset=['rental_price']).reset_index(drop=True)

y = df['rental_price']
X = df.drop(columns=['rental_price'])

# Drop common identifier/free-text columns if present (avoids huge one-hot)
drop_if_present = [
    'id','ID','listing_id','property_id','address','url','link','description','title',
    'full_address','street_address','listing_url','latlon_str','geometry','geom','wkt'
]
cols_to_drop = [c for c in drop_if_present if c in X.columns]
if cols_to_drop:
    X = X.drop(columns=cols_to_drop)

print('Feature matrix shape:', X.shape, '| Target length:', len(y))
X.head(3)


Feature matrix shape: (2899, 316) | Target length: 2899


,bedrooms,bathrooms,car_spaces,property_type,suburb,year,quarter,longitude,latitude,closest_ptv_station_id,...,best_school_name_walking_10min,best_school_coord_walking_10min,best_score_walking_10min,best_dist_km_walking_10min,n_schools_walking_10min,best_school_name_walking_15min,best_school_coord_walking_15min,best_score_walking_15min,best_dist_km_walking_15min,n_schools_walking_15min
0,4,3,2,house,truganina,2025,3,144.689994,-37.770750,NaN,...,Bemin Secondary College,POINT (144.70975 -37.82641),0.062926,6.418188,23.0,NaN,NaN,3.000000e-08,6.245875,0.0
1,2,1,2,House,croydon-lilydale,2025,1,145.338663,-37.757202,NaN,...,Lilydale High School,POINT (145.34392 -37.7607),-0.000089,0.604427,2.0,Lilydale High School,POINT (145.34392 -37.7607),-8.921519e-05,0.604427,3.0
2,4,2,2,house,werribee-hoppers crossing,2025,3,144.635706,-37.921006,51894,...,Werribee Secondary College,POINT (144.66874 -37.90554),-0.000060,3.374128,2.0,NaN,NaN,3.000000e-08,6.245875,2.0


In [34]:
# Identify feature types

# set categorical features
categorical_features = [
    'property_type', 'suburb'
]

# set numerical features
numeric_features = [
    'bedrooms', 'bathrooms', 'car_spaces',
    'year', 'quarter',
    'best_score_walking_5min', 'best_score_walking_10min', 
    'best_score_walking_15min', 'best_dist_km_driving_5min', 'best_dist_km_driving_10min',
    'best_dist_km_driving_15min', 'best_dist_km_walking_5min', 'best_dist_km_walking_10min',
    'best_dist_km_walking_15min', 'n_schools_driving_5min', 'n_schools_driving_10min',
    'n_schools_driving_15min', 'n_schools_walking_5min', 'n_schools_walking_10min',
    'n_schools_walking_15min', 'missing_iso_feature'
]

print(f"Categorical: {len(cat_cols)} | Numeric: {len(num_cols)}")

Categorical: 7 | Numeric: 37


In [35]:
# quickly clean property_type
# was missed in earlier cleaning steps
if 'property_type' in X.columns:
    X['property_type'] = X['property_type'].str.lower().str.strip()
    common_types = ['house', 'unit', 'apartment', 'townhouse', 'villa', 'duplex']
    X['property_type'] = X['property_type'].where(
        X['property_type'].isin(common_types), other='other'
    )
    print("Unique property_type values after cleaning:", X['property_type'].unique())

Unique property_type values after cleaning: ['house' 'other' 'townhouse' 'villa' 'duplex']


In [36]:
# -- Ignore --
# Identify feature types with a guard against high-cardinality categoricals
# numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
# categorical_candidates = X.columns.difference(numeric_features)

# max_unique_cats = 100  # avoid exploding OHE on huge text-like columns
# categorical_features, dropped_high_cardinality = [], []
# for col in categorical_candidates:
#     nunique = X[col].nunique(dropna=True)
#     if nunique <= max_unique_cats:
#         categorical_features.append(col)
#     else:
#         dropped_high_cardinality.append(col)

# print(f"Numeric features: {len(numeric_features)}")
# print(f"Categorical features (<= {max_unique_cats} levels): {len(categorical_features)}")
# if dropped_high_cardinality:
#     print(f"Dropping {len(dropped_high_cardinality)} high-cardinality columns (sample): {dropped_high_cardinality[:10]}")


In [37]:
# Build preprocessors
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=True)),
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ],
    remainder='drop',
    verbose_feature_names_out=True,
)

# Train/val/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Fit preprocessor on training data and transform all splits
preprocessor.fit(X_train)
X_train_t = preprocessor.transform(X_train)
X_val_t = preprocessor.transform(X_val)
X_test_t = preprocessor.transform(X_test)

print('Transformed shapes:', X_train_t.shape, X_val_t.shape, X_test_t.shape)


Transformed shapes: (1855, 342) (464, 342) (580, 342)


In [38]:
# Define and train the XGBoost regressor (with early stopping on the validation set)
xgb = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1,
    tree_method='hist',
    eval_metric='rmse',
)

fit_kwargs = {
    'eval_set': [(X_val_t, y_val)],
}
# Prefer callbacks-based early stopping when available (XGBoost >=2.x/3.x)
if XGB_EarlyStopping is not None:
    fit_kwargs['callbacks'] = [XGB_EarlyStopping(rounds=50, save_best=True, maximize=False)]
else:
    # Fallback for older versions
    fit_kwargs['early_stopping_rounds'] = 50

xgb.fit(X_train_t, y_train, **fit_kwargs)

print('Best iteration:', getattr(xgb, 'best_iteration', None))


TypeError: XGBModel.fit() got an unexpected keyword argument 'callbacks'

In [ ]:
# Evaluate performance
def regression_metrics(y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return {'RMSE': rmse, 'MAE': mae, 'R2': r2}

y_train_pred = xgb.predict(X_train_t)
y_val_pred = xgb.predict(X_val_t)
y_test_pred = xgb.predict(X_test_t)

print('Train metrics:', regression_metrics(y_train, y_train_pred))
print('Val metrics:  ', regression_metrics(y_val, y_val_pred))
print('Test metrics: ', regression_metrics(y_test, y_test_pred))

plt.figure(figsize=(6, 6))
sns.scatterplot(x=y_test, y=y_test_pred, s=12, alpha=0.5)
lims = [min(y_test.min(), y_test_pred.min()), max(y_test.max(), y_test_pred.max())]
plt.plot(lims, lims, 'r--', linewidth=1)
plt.xlabel('Actual rental_price')
plt.ylabel('Predicted rental_price')
plt.title('Predicted vs Actual (Test)')
plt.show()


In [ ]:
# Optional: cross-validation on the full dataset (may be slow on large files)
DO_CV = False
if DO_CV:
    model = Pipeline(steps=[('preprocessor', preprocessor), ('model', xgb)])
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X, y, cv=kf, scoring='neg_root_mean_squared_error', n_jobs=-1)
    print('CV RMSE: ', (-cv_scores).mean(), '+/-', (-cv_scores).std())


In [ ]:
# Feature importances
importances = xgb.feature_importances_
feature_names = preprocessor.get_feature_names_out()
feat_imp = pd.DataFrame({'feature': feature_names, 'importance': importances}).sort_values('importance', ascending=False)
feat_imp.head(20)

top_n = 20
plt.figure(figsize=(8, max(4, top_n * 0.4)))
sns.barplot(data=feat_imp.head(top_n), x='importance', y='feature', orient='h')
plt.title(f'Top {top_n} Feature Importances (XGBoost)')
plt.tight_layout()
plt.show()


In [ ]:
# Optional: save fitted pipeline (preprocessing + model)
from joblib import dump
save_dir = Path('models')
save_dir.mkdir(parents=True, exist_ok=True)
fitted_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', xgb)])
model_path = save_dir / 'xgb_rental_price_pipeline.joblib'
dump(fitted_pipeline, model_path)
print(f'Saved pipeline to {model_path}')
